In [1]:
import backtrader as bt
import datetime
import pandas as pd
import math

In [2]:
# import pymysql


# # Connect to the database
# connection = pymysql.connect(host='localhost',
#                              port=3306,
#                              user='root',
#                              password='seiRaefoe9jeufooT1uipei5gungiFah',
#                              db='stock_etf',
#                              charset='utf8mb4',
#                              cursorclass=pymysql.cursors.SSDictCursor)


# try:
#     with connection.cursor() as cursor:
#         # Read a single record
#         sql = "SELECT * FROM `etf159920_15m` WHERE datetime BETWEEN %s AND %s"
#         cursor.execute(sql, (datetime.date(2019,1,1),datetime.date(2020,5,1)) )
#         result = cursor.fetchall()
# finally:
#     connection.close()

# df = pd.DataFrame(data=result)
# df = df.set_index(['datetime'])
# df

In [3]:
from lutils.stock import LTdxHq

In [4]:
ltdxhq = LTdxHq()

In [5]:
# ll = ltdxhq.stock_list()

In [6]:
# ll[(ll['pre_close'] > 30) & (ll['pre_close'] < 50)]

In [7]:
# 399300
code = '603636' # 000032 300142 603636 600519 688567
# df = ltdxhq.get_k_data_daily('300142') # 000032 300142 603636 600519
# df = ltdxhq.get_k_data_15min(code) #' # 000032 300142 603636 600519
# df = ltdxhq.to_qfq(code, df)

In [8]:
# df.index = df.index.unique(level=1)

In [9]:
df15 = ltdxhq.get_k_data_15min(code) #' # 000032 300142 603636 600519
df15 = ltdxhq.to_qfq(code, df15)

df60 = ltdxhq.get_k_data_1hour(code) #' # 000032 300142 603636 600519
df60 = ltdxhq.to_qfq(code, df60)

df1d = ltdxhq.get_k_data_daily(code) #' # 000032 300142 603636 600519
df1d = ltdxhq.to_qfq(code, df1d)

In [10]:
df15.index = df15.index.unique(level=1)
df60.index = df60.index.unique(level=1)
df1d.index = df1d.index.unique(level=0)

In [11]:
ltdxhq.close()

In [12]:
df15.index = pd.to_datetime(df15.index)
df60.index = pd.to_datetime(df60.index)
df1d.index = pd.to_datetime(df1d.index)

In [13]:
data15 = df15['2021-01-01':]
data60 = df60['2021-01-01':]
data1d = df1d['2021-01-01':]

In [14]:
data1d

,open,close,high,low,volume,amount
date,,,,,,
2021-01-04,9.179221,9.287792,9.297662,9.090390,4484555.0,41926596.0
2021-01-05,9.307532,9.317403,9.435844,9.277922,3896723.0,36895040.0
2021-01-06,9.258182,9.159481,9.297662,9.110130,3874385.0,36067080.0
2021-01-07,9.149610,8.843636,9.149610,8.685714,5851525.0,52623452.0
2021-01-08,8.804156,8.912727,9.120000,8.646234,4604155.0,41699760.0
...,...,...,...,...,...,...
2021-09-27,8.920000,8.640000,8.970000,8.620000,5689958.0,49751368.0
2021-09-28,8.640000,8.550000,8.650000,8.490000,3466596.0,29651572.0
2021-09-29,8.520000,8.280000,8.520000,8.230000,4925307.0,41109280.0


In [15]:
class MultiTFStrategy(bt.Strategy):
    params = (
        ('macd1', 12),
        ('macd2', 26),
        ('macdsig', 9),
        ('atrperiod', 14),  # ATR Period (standard)
        ('atrdist', 3.0),   # ATR distance for stop price
        ('smaperiod', 30),  # SMA Period (pretty standard)
        ('dirperiod', 10),  # Lookback period to consider SMA trend direction
        
        ('short', 30),
        ('long', 70),
    )
    
    # states defination
    Empty, M15Hold, H1Hold, D1Hold = range(4)
    States = [
        'Empty', 'M15Hold', 'H1Hold', 'D1Hold',
    ]
    
    def log(self, txt):
        ''' Logging function for this strategy'''
        dt = self.datas[0].datetime.datetime(0)
        print('%s, %s' % (dt.isoformat(), txt))
        
    def __init__(self):

        self.rsi15 = bt.indicators.RSI_EMA(self.dnames.hs15m.close, period=14) # RSI_SMA
        self.rsi60 = bt.indicators.RSI_EMA(self.dnames.hs1h.close, period=14)
        self.rsi1d = bt.indicators.RSI_EMA(self.dnames.hs1d.close, period=14)
    
        self.st = self.Empty
        self.st_map = {
            self.Empty: self._empty,
            self.M15Hold: self._m15hold,
            self.H1Hold: self._h1hold,
            self.D1Hold: self._d1hold,
        }
        
        # To keep track of pending orders
        self.order = None
        
    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status == order.Completed:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, St: %s, Size: %d, Price: %.2f, Cost: %.2f, Comm %.2f' %
                    (
                        self.States[self.st],
                        order.executed.size,
                        order.executed.price,
                        order.executed.value,
                        order.executed.comm,
                    )
                )

            else:  # Sell
                self.log(
                    'SELL EXECUTED, St: %s, Size: %d, Price: %.2f, Cost: %.2f, Comm %.2f' %
                    (
                        self.States[self.st],
                        order.executed.size,
                        order.executed.price,
                        order.executed.value,
                        order.executed.comm
                    )
                )

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        # Write down: no pending order
        self.order = None
    
    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))

    def next(self):
        # Check if an order is pending ... if yes, we cannot send a 2nd one
        if self.order:
            return
        
        # just call state_map function
        self.order = self.st_map[self.st]()
        
        # Check if we are in the market and no buy order issued
        if self.position and not self.order:
            # Already in the market ... we might sell
            if self.rsi1d > self.params.long:
                self.st = self.Empty
                # Keep track of the created order to avoid a 2nd order
                self.order = self.close()
                
    def _empty(self):
        if self.rsi15 < self.params.short:
            price = self.data0.close[0]
            cash = self.broker.get_cash()
            # 20% of the cash
            share = int(math.floor((0.2*cash)/price))

            # set state
            self.st = self.M15Hold
            return self.buy(size=share)
        
    def _m15hold(self):
        if self.rsi60 < self.params.short:
            price = self.data0.close[0]
            cash = self.broker.get_cash()
            # half of the remain cash ( 60% )
            share = int(math.floor((0.6*cash)/price))
            
            # set state
            self.st = self.H1Hold
            return self.buy(size=share)
        
    def _h1hold(self):
        if self.rsi1d < self.params.short:
            price = self.data0.close[0]
            cash = self.broker.get_cash()
            # half of the remain cash (80%)
            share = int(math.floor((0.8*cash)/price))
            
            # set state
            self.st = self.D1Hold
            return self.buy(size=share)
        
    def _d1hold(self):
        return None

In [16]:
cerebro = bt.Cerebro(oldtrades=True)

feed15 = bt.feeds.PandasData(dataname=data15, openinterest=None, compression=15, timeframe=bt.TimeFrame.Minutes)
feed60 = bt.feeds.PandasData(dataname=data60, openinterest=None, compression=60, timeframe=bt.TimeFrame.Minutes)
feed1d = bt.feeds.PandasData(dataname=data1d, openinterest=None, compression=1, timeframe=bt.TimeFrame.Days)

cerebro.adddata(feed15, name='hs15m')
cerebro.adddata(feed60, name='hs1h')
cerebro.adddata(feed1d, name='hs1d')

# cerebro.adddata(feed, name='hs1d')
# cerebro.resampledata(feed, name='hs1h', timeframe=bt.TimeFrame.Minutes, compression=60)
# cerebro.resampledata(feed, name='hs1d', timeframe=bt.TimeFrame.Days)

cerebro.addstrategy(MultiTFStrategy)

# 小场面1万起始资金
cerebro.broker.setcash(10000.0)

# 手续费万5
cerebro.broker.setcommission(0.0005)

print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

result = cerebro.run()

print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

Starting Portfolio Value: 10000.00
2021-01-22T10:00:00, BUY EXECUTED, St: M15Hold, Size: 268, Price: 7.46, Cost: 1998.79, Comm 1.00
2021-01-22T10:45:00, BUY EXECUTED, St: H1Hold, Size: 645, Price: 7.44, Cost: 4800.07, Comm 2.40
2021-02-19T09:45:00, SELL EXECUTED, St: Empty, Size: -913, Price: 8.09, Cost: 6798.86, Comm 3.69
2021-02-19T09:45:00, OPERATION PROFIT, GROSS 587.13, NET 580.04
2021-02-22T14:45:00, BUY EXECUTED, St: M15Hold, Size: 256, Price: 8.25, Cost: 2112.45, Comm 1.06
2021-03-09T09:45:00, BUY EXECUTED, St: H1Hold, Size: 624, Price: 8.19, Cost: 5108.67, Comm 2.55
2021-04-22T09:45:00, SELL EXECUTED, St: Empty, Size: -880, Price: 8.39, Cost: 7221.13, Comm 3.69
2021-04-22T09:45:00, OPERATION PROFIT, GROSS 161.57, NET 154.27
2021-04-23T13:30:00, BUY EXECUTED, St: M15Hold, Size: 255, Price: 8.41, Cost: 2143.43, Comm 1.07
2021-04-27T10:45:00, BUY EXECUTED, St: H1Hold, Size: 626, Price: 8.24, Cost: 5155.46, Comm 2.58
2021-04-28T09:45:00, BUY EXECUTED, St: D1Hold, Size: 329, Price:

In [17]:
cerebro.plot(
    iplot=False,
    start=datetime.date(2021, 1, 1),
    end=datetime.date(2021, 9, 30),
    style='bar',
    barup='red',
    bardown='green',
)

[[<Figure size 640x480 with 11 Axes>]]